# Init


In [ ]:
# Install necessary requirements
%pip install -r ../requirements.txt

# Change working directory to root
import os
if os.getcwd().endswith("notebooks"):
    %cd ..
    print(os.getcwd())

# Automatically reload changes in code
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from typing import List
import pandas as pd
from association_finder.concept_drifts_finder import ConceptDriftsFinder
from association_finder.models import Transaction, ConceptDriftResult
from association_finder.concept_engineering import ConceptEngineering
from sklearn.model_selection import train_test_split
from association_finder.preprocessing import preprocess_dataset, split_X_y
from typing import Dict, Tuple, Optional
from dataclasses import dataclass
import numpy as np
from sklearn.linear_model import LogisticRegression


logging.basicConfig(level=logging.INFO)

# Read and parse file

In [ ]:
np.random.seed(0)

# Read file
train_dataset_path = "datasets/houseprices/train.csv"
df = pd.read_csv(train_dataset_path, index_col='Id')

# Split
df_train, df_val = train_test_split(df, test_size=0.3)

# Preprocess    
df_train_prep, train_params = preprocess_dataset(df_train)

# Focusing on prominent columns:
good_columns = [column for column in ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'OverallCond', 'BldgType', 'LotArea',
                'GrLivArea', 'FullBath', 'BedroomAbvGr', 'LotFrontage', 'TotalBsmtSF', 'SalePrice'] if column not in 
                train_params.dropped_columns]
target_column = "SalePrice"
one_hot_columns = [column for column in ['BldgType'] if column not in train_params.dropped_columns]


In [ ]:
# Prepare data for training
X_train, y_train = split_X_y(df_train_prep, good_columns, train_params, one_hot_columns, target_column)
X_val, y_val = split_X_y(preprocess_dataset(df_val, train_params)[0], good_columns, train_params, one_hot_columns, target_column)

# Baseline

In [ ]:
# Train baseline model
clf = LogisticRegression(random_state=0, max_iter=10000).fit(X_train, y_train)

In [ ]:
print(f"Train accuracy: {clf.score(X_train, y_train)}")
print(f"Validation accuracy: {clf.score(X_val, y_val)}")

# Expected:
# Train accuracy: 0.6976516634050881
# Validation accuracy: 0.6598173515981736

# Build model using rules

In [ ]:
# Find association rules
concept_engineering = ConceptEngineering()
X_train_rules = concept_engineering.fit_transform(X_train, df_train_prep[good_columns], target_column, one_hot_columns)
X_val_rules = concept_engineering.transform(X_val)

In [ ]:
# Train model using association rules
clf_rules = LogisticRegression(random_state=0, max_iter=10000).fit(X_train_rules, y_train)

In [ ]:
print(f"Train accuracy: {clf_rules.score(X_train_rules, y_train)}")
print(f"Validation accuracy: {clf_rules.score(X_val_rules, y_val)}")

# Previously:
# Train accuracy: 0.6966731898238747
# Validation accuracy: 0.6666666666666666

# Expected after fixing weight:
# Train accuracy: 0.6986301369863014
# Validation accuracy: 0.6689497716894978

# Analyze concepts

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
concept_engineering.concepts_df

### Error analysis

In [ ]:
pred_df = pd.DataFrame(zip(clf_rules.predict(X_train_rules), y_train), columns=['predict', 'actual'], index=X_train.index)
pred_df = pd.merge(pred_df, X_train_rules, left_index=True, right_index=True)
errors_df = pred_df[pred_df['predict'] != pred_df['actual']]
errors_df[:50]

### Model coefficients analysis

In [ ]:
list(enumerate(sorted(list(zip(clf.coef_[3], X_train.columns)))))

In [ ]:
list(enumerate(sorted(list(zip(clf_rules.coef_[3], X_train.columns)))))